In [7]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chat = ChatOpenAI(temperature=0.1)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

In [8]:
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'